In [1]:
# David Mao, dm46452
# Stephen Aigbomian, sa46979

## Part 1 - Apriori Algorithm

Apriori algorithm is a classical algorithm in data mining. It is used for mining frequent itemsets and relevant association rules. In this part, you'll be implementing this algorithm for generating the itemsets that occur more than the `min_sup` threshold. Based on these frequent itemsets you'll find association rules that have confidence above the `min_conf` threshold.

# Association Analysis

Association analysis uses machine learning to extract hidden relationships from large datasets. In this assignment you'll be implementing two of the most commonly used algorithms for association rule mining: Apriori and FP-Growth.

The dataset (`large_retail.txt`) that we are going to use for this assignment has been adapted from the [Retail Market Basket Dataset](http://fimi.ua.ac.be/data/retail.pdf). The dataset contains transaction records supplied by an anonymous Belgian retail supermarket store. Each line in the file represents a separate transaction with the item ids separated by space. The dataset has 3000 transaction records and 99 different item ids.

We also provide a smaller dataset (`small_retail.txt`) with 9 transactions and 5 different item ids along with the solutions. *You should first test your implementation on this dataset, before running it on the larger dataset.*

The assignment will be **autograded**. We will use the `diff` command in linux to compare the output files. So please **check your answers** based on the given sample output files.

Implementation Hint:

- Use the `frozenset` data structure in Python (similar to `set` in functionality) to represent the itemsets because `frozenset` is a hashable data structure. You can maintain a dictionary that maps from the itemset (a `frozenset`) to its support count.

In [2]:
# Standard imports (you can add additional headers if you wish)
import numpy as np

In [3]:
# Reading the dataset from file
def load_dataset(file_name):
    with open(file_name, 'r') as f:
        content = f.readlines()
        data = [[int(x) for x in line.rstrip().split()] for line in content]
    return data

In [4]:
small_dataset = load_dataset('small_retail.txt')
large_dataset = load_dataset('large_retail.txt')

**Q1.** Implement the function `create_1_itemsets` that takes as input the entire dataset and returns a list of all the 1-itemsets. For example, for `small_retail.txt` it should return:
~~~
[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]
 ~~~
 Please **don't hardcode** the item ids, your code should support item ids that are non-sequential.

In [5]:
def create_1_itemsets(dataset):
    c1 = set()
    
    for items in dataset:
        for item in items:
            c1.add(frozenset([item]))
    return list(c1)

**Q2.** Implement function `filter_candidates` that takes as input the candidate itemsets, the dataset, and the minumum support count `min_sup`, and filters out candidates that don't meet the support threshold.

**Hint:** You should also return the support count information (perhaps as a `dict`) for the itemsets. This will be useful later on for the final output.

In [6]:
def filter_candidates(candidates, dataset, min_sup):
    retlist = []
    support_data = {}
    for candidate in candidates:
        for items in dataset:
            if candidate <= set(items): # maybe need to do set(items)
                if candidate in support_data:
                    support_data[candidate] += 1
                else:
                    support_data[candidate] = 1
    
    filtered_support_data = {}
    for candidate, support_count in support_data.items():
        if support_count >= min_sup:
            retlist.append(candidate)
            filtered_support_data[candidate] = support_count
    
    return retlist, filtered_support_data

**Q3.** Implement the function `generate_next_itemsets` that takes in frequent itemsets of size `k` and generates candidate itemsets of size `k + 1`.

**Hint:** Use the fact that if `[1, 2, 3, 4]` is a frequent itemset of size 4 then `[1, 2, 3]` and `[1, 2, 4]` both will be frequent itemsets of size 3. You can use this to drastically reduce the number of candidate itemsets that you need to generate. (Use the F(k-1) x F(k-1) or F(k-1) x F(1) candidate generation method).

In [7]:
def generate_next_itemsets(freq_sets):
    retlist = set()
    
    for i in range(len(freq_sets)):
        cur_item_set = freq_sets[i]
        for j in range(i + 1, len(freq_sets)):
            next_item_set = freq_sets[j]
            candidate = cur_item_set | next_item_set
            if len(candidate) == len(cur_item_set) + 1:
                retlist.add(candidate)
    return list(retlist)

**Q4.** Implement the function `apriori_freq_temsets` that takes the entire dataset as the input and returns the frequent itemsets that have support count more than `min_sup`.

In [8]:
def apriori_freq_itemsets(dataset, minsup):
     # your code goes here
    freq_itemsets = []
    cur_itemsets = create_1_itemsets(dataset)
    cur_itemsets, support_data = filter_candidates(cur_itemsets, dataset, minsup)
    while cur_itemsets:
        freq_itemsets += cur_itemsets
        candidates = generate_next_itemsets(cur_itemsets)
        cur_itemsets, cur_sup_data = filter_candidates(candidates, dataset, minsup)
        support_data.update(cur_sup_data)
    return freq_itemsets, support_data

**Q5.** Display the frequent item sets in the form of a table along with their `support_fraction` for the `large_retail.txt` dataset with min support count 300.

Sample Table Format (tab separated table)

~~~
Sup     Freq Itemset
0.67	[1]
0.44	[1, 2]
(and so on)
...
...
~~~

`support_fraction(itemset) = support_count(itemset) / num_total_transactions`.

Note that the `support_fraction` should be rounded to the nearest 2 decimal places (use `round(sup, 2)`). Also `support_fraction` and the itemset should be separated by a tab (`'\t'`). The itemsets should also be in a sorted order where smaller itemsets should come before larger itemsets and itemsets of the same size should be sorted amongst themselves.

For eg. 
~~~~
[1, 2] should come before [1, 2, 3]
[1, 2, 3] should come before [1, 2, 4]
[1, 2, 3] should come before [1, 4, 5]
[1, 2, 3] should come before [2, 3, 4]
~~~~

Note that **this order is very important** because your output will be checked using the `diff` command. The output also **shouldn't contain any duplicates**. The sample output for the `small_retail.txt` dataset with min support count as 2 is:

~~~~
Sup     Freq Itemset
0.67	[1]
0.78	[2]
0.67	[3]
0.22	[4]
0.22	[5]
0.44	[1, 2]
0.44	[1, 3]
0.22	[1, 5]
0.44	[2, 3]
0.22	[2, 4]
0.22	[2, 5]
0.22	[1, 2, 3]
0.22	[1, 2, 5]
~~~~

**Store** this output for the `large_retail.txt` dataset in the file `apriori_itemsets.txt`. The sample output file for the `small_retail.txt` dataset has been provided to you as `small_apriori_itemsets.txt` for your convenience.

In [9]:
# your code goes here

def format_itemsets(freq_itemsets, support_count):
    formatted_itemsets = []
    for itemset in freq_itemsets:
        sup_count = support_count[itemset]
        formatted_itemset = sorted(itemset)
        formatted_itemsets.append((formatted_itemset, sup_count))
    
    formatted_itemsets.sort(key=lambda itemset: (len(itemset[0]), itemset[0]))
    return formatted_itemsets

def print_support(freq_itemsets, num_transactions, fi):
    print("Sup\tFreq Itemset", file=fi)
    for itemset, count in freq_itemsets:     
        sup = count / num_transactions
        print("{}\t{}".format(round(sup, 2), sorted(itemset)), file=fi)

# apriori_freq_itemsets(small_dataset, 2)
freq_itemsets, sup_data = apriori_freq_itemsets(large_dataset, 300)
formatted_itemsets = format_itemsets(freq_itemsets, sup_data)
with open('apriori_itemsets.txt', 'w') as fi:
    print_support(formatted_itemsets, len(large_dataset), fi)

**Q6.** Find the closed frequent item sets along with their `support_fraction`. Store the results for the `large_retail.txt` dataset in the file `apriori_closed_itemsets.txt`, in the same format as specified in Q5.

In [10]:
# your code goes here
def find_closed_itemsets(freq_itemsets, sup_data):
    closed = set()
    for itemset in freq_itemsets:
        cur_closed = True
        for closed_itemset in closed:
            # subset check
            if itemset <= closed_itemset and sup_data[itemset] <= sup_data[closed_itemset]:
                cur_closed = False
        # add ourself to closed itemset, do superset check
        if cur_closed:
            closed.add(itemset)
            removed_subsets = set()
            # find the subsets to remove
            for closed_itemset in closed:
                if itemset > closed_itemset and sup_data[itemset] == sup_data[closed_itemset]:
                    removed_subsets.add(closed_itemset)
            closed -= removed_subsets
                
    return closed

closed_itemsets = find_closed_itemsets(freq_itemsets, sup_data)
formatted_closed_itemsets = format_itemsets(closed_itemsets, sup_data)
with open('apriori_closed_itemsets.txt', 'w') as fi:
    print_support(formatted_closed_itemsets, len(large_dataset), fi)

## Part 2 - Frequent Pattern Tree

The FP-Growth Algorithm, proposed by [Han](https://www.cs.sfu.ca/~jpei/publications/sigmod00.pdf), is an efficient and scalable method for mining the complete set of frequent patterns by pattern fragment growth, using an extended prefix-tree structure for storing compressed and crucial information about frequent patterns named frequent-pattern tree (FP-tree).

Use the following FPTreeNode class to build your FP Tree. You may add methods to it if you wish.

In [11]:
# variables:
# self.item_id: item id of the item
# self.item_count: item count for this node
# self.node_link: next pointer for the linked list that connects
#                 nodes of the same item_id (required for the FP-growth algorithm)
# self.parent: pointer to the parent node (should be None for the root)
# self.children: dictionary for the children (maps from item_id to the FPTreeNode object)
# NOTE: root node should have item_id as -1
class FPTreeNode:
    def __init__(self, uid, num):
        self.item_id = uid
        self.item_count = num
        self.node_link = None
        self.parent = None
        self.children = {}
        
    def insert_node(self, transaction, first_linked):
        cur_node = self
        for item in transaction:
            if item in cur_node.children:
                cur_node = cur_node.children[item]
                cur_node.item_count += 1
            else:
                child_node = FPTreeNode(item, 1)
                child_node.parent = cur_node
                cur_node.children[item] = child_node
                if item in first_linked:
                    insert_node_link(first_linked[item], child_node)
                else:
                    first_linked[item] = child_node
                cur_node = child_node
        
    def displayTree(self, tab=1):
        if self.item_id == -1:
            print ('  '*tab, 'root')
        else:
            print ('  '*tab, 'item_id:', self.item_id, 'item_count:', self.item_count)
        for key in sorted(self.children.keys()):
            self.children[key].displayTree(tab + 1)
            
    # helper function for saveToFile
    def saveToFile_helper(self, fp, tab=1):
        if self.item_id == -1:
            print ('  '*tab, 'root', file=fp)
        else:
            print ('  '*tab, 'item_id:', self.item_id, 'item_count:', self.item_count, file=fp)
        for key in sorted(self.children.keys()):
            self.children[key].saveToFile_helper(fp, tab + 1)
    
    # call this to save to file
    def saveToFile(self, filename):
        with open(filename, 'w') as fp:
            self.saveToFile_helper(fp)

def insert_node_link(cur_node, new_node):
    while cur_node.node_link:
        cur_node = cur_node.node_link
    cur_node.node_link = new_node


**Q7.** Write the function `order_transactions` that takes in the dataset and reorders the items in each transaction based on their frequency (or support counts) and also removes any items that do not meet minsup. This function should return the reordered transactions as well as a dictionary containing the support counts of the frequent 1-itmesets.

Note that **item_ids that have equal frequency should be ordered based on the item_id value**.

For example,
~~~~
For the small_retail.txt dataset:
{item_id: frequency} -> {1: 6, 2: 7, 3: 6, 4: 2, 5: 2}
The transaction [1, 2, 3, 5] should be reordered as [2, 1, 3, 5]
Notice the ordering of 1 and 3 (both have 6 occurences in the dataset)

If minsup=3, [1, 2, 3, 5] should be reordered as [2, 1, 3] because 5 does not meet minsup.
~~~~

Note that the **relative order between transactions should not be changed**, they remain in the same order as they appear in the original dataset. For the `small_retail.txt` the transactions would be inserted in the FP Tree in this order:
~~~~
[2, 1, 5]
[2, 4]
[2, 3]
[2, 1, 4]
[1, 3]
[2, 3]
[1, 3]
[2, 1, 3, 5]
[2, 1, 3]
~~~~

In [12]:
def order_transactions(dataset, minsup):
    freq_cnt = {}
    new_dataset = []
    for transaction in dataset:
        for item in transaction:
            if item not in freq_cnt:
                freq_cnt[item] = 0
            freq_cnt[item] += 1
    for transaction in dataset:
        sorted_transaction = sorted(transaction, key=lambda id: freq_cnt[id], reverse=True)
        while sorted_transaction and freq_cnt[sorted_transaction[-1]] < minsup:
            sorted_transaction.pop()
        if sorted_transaction:
            new_dataset.append(sorted_transaction)
    # your code goes here
    return new_dataset, freq_cnt

**Q8.** Implement the function `build_FP_tree` which takes the ordered itemsets from Q7 and inserts them into the FP Tree. Build the FP Tree for the `large_retail.txt` dataset and write the resulting tree out to `large_fp_tree.txt`.

The tree output for `small_retail.txt` dataset is given as follows:
~~~~
   root
     item_id: 1 item_count: 2
       item_id: 3 item_count: 2
     item_id: 2 item_count: 7
       item_id: 1 item_count: 4
         item_id: 3 item_count: 2
           item_id: 5 item_count: 1
         item_id: 4 item_count: 1
         item_id: 5 item_count: 1
       item_id: 3 item_count: 2
       item_id: 4 item_count: 1
~~~~
This output has been provided to you as `small_fp_tree.txt` for your convenience. You can use the `diff` command in Linux to check your output with the provided output.




In [13]:
def build_fp_tree (ordered_transactions):
    """
    Builds an FP tree; importantly, links nodes with the same item_id and returns the first 
    node for each item_id in the first_linked_nodes dictionary
    
    parameters:
        ordered_transactions: the transactions with items sorted in descending rate of occurence

    returns:
        root (FPTreeNode): the root node of the FP tree
        first_linked_nodes (dict of int: FPTreeNode): a map from an item_id to first node of that
                                                        item_id; following each node's node_link should 
                                                        yield every node with that item_id
    """        
    root = FPTreeNode(-1, 0)
    first_linked_nodes = {}
    
    # Your code here
    for transaction in ordered_transactions:
        root.insert_node(transaction, first_linked_nodes)
    return root, first_linked_nodes


**Q9.** Implement `build_cond_pat_base` which take in an FPTreeNode (representing an item) and builds its conditional pattern base. This essentially enumerates all paths in the tree that contain item's item_id.

The item (FPTreeNode) that gets passed in to this function will be an FPTreeNode in your header table. This function will first find the path leading to this node. Then it will move to the next FPTreeNode with this same item_id by using `item.node_link` and find the path leading to that node. It will iterate over all items with this item_id by using `item.node_link`.

NOTE: The conditional pattern base does not contain item itself. 

Return the conditional pattern base as a dictionary of tuples to ints representing the path to the item an how many of that item this path gets you to. For example, in the conditional pattern base for item_id 5 in `small_retail.txt` would be: `{(2, 1): 1, (2, 1, 3): 1}`

In [14]:
from collections import deque
def build_cond_pat_base(item):
    """
    Builds a conditional pattern base for an item, starting with the first FPTreeNode of this item 
    and traversing the node_links to reach all other FPTreeNodes of this item.
    
    parameters:
        item (FPTreeNode): the node we are building a conditional pattern base for
    returns:
        cond_pat_base (dict of tuple of int: int): conditional pattern base; the patterns with which item's 
                                                   item_id is found in transactions with and their associated
                                                   counts
    """
    cond_pat_base = {}
    # your code goes here
    while item:
        count = item.item_count
        cur_node = item.parent
        cond_pat = deque()
        while cur_node.item_id > 0:
            assert cur_node.item_count >= count
            cond_pat.appendleft(cur_node.item_id)
            cur_node = cur_node.parent
            
        cond_pat_tuple = tuple(cond_pat)
        assert cond_pat_tuple not in cond_pat_base
        if cond_pat_tuple:
            cond_pat_base[cond_pat_tuple] = count
        item = item.node_link
    return cond_pat_base

**Q10.** Implement the function `is_single_path` that takes in an FP-Tree root node and determines whether or not it is a single-path tree (no branches). Return True for a single-path tree, and False otherwise.

In [15]:
def is_single_path(fp_tree):  
    # your code goes here
    while fp_tree:
        if len(fp_tree.children) > 1:
            return False
        elif len(fp_tree.children) == 0:
            return True
        else:
            for child, value in fp_tree.children.items():
                fp_tree = value

**Q11.** Now, implement the FP Growth algorithm to mine the FP-Tree for frequent patterns. 

Starting from the least frequent 1-itemsets:
    1. build the conditional pattern base
    2. build a new FP tree from the conditional pattern base
You then need to handle the following cases:
    1. the newly constructed FP tree has no children
    2. the newly constructed FP tree is a single path tree
    3. the newly constructed FP tree is not a single path tree
    
    If the new FP tree is not a single path tree, you will need to recurse.
    If the new FP tree is a single path tree, you will need to enumerate the subsets of the path.

You may add addition helper functions as you see fit. But remember you are only allowed to mine the FP tree that you generate in Q8. You *cannot use the dataset* other than in the first line of code provided here.

**Store** the output of mining the `large_retail.txt` dataset with a minsup=300 in the file `fp_growth_itemsets.txt`.

Hint: You can check your answer by comparing it to the output of your Apriori algorithm.

In [16]:
def enumerate_subsets(fp_tree):
    if not fp_tree.children:
        return []
    else:
        path = []
        while fp_tree.children:
            assert len(fp_tree.children) == 1
            min_count = float("inf")
            for child, value in fp_tree.children.items():
                path.append(child)
                min_count = min(min_count, value.item_count)
                fp_tree = value
        # generate all subsets
        return list(yield_subsets(path, min_count))

def yield_subsets(pattern, min_count):
    if len(pattern) == 1:
        yield pattern, min_count
    else:
        for index, val in enumerate(pattern):
            yield [val], min_count
            for subset, min_count in list(yield_subsets(pattern[index + 1:], min_count)):
                yield [val] + subset, min_count
        
def yield_frequent_patterns(fp_tree, link_table, minsup, freq_items):
    if is_single_path(fp_tree):
        yield from enumerate_subsets(fp_tree)
    else:
        for item, links in link_table.items():
            cond_pat_base = build_cond_pat_base(links)
            transactions = []
            for pattern, count in cond_pat_base.items():
                for i in range(count):
                    transactions.append(pattern)
            ordered_transactions, new_freq_items = order_transactions(transactions, minsup)
            new_fp_tree, new_link_table = build_fp_tree(ordered_transactions)
            yield [item], freq_items[item]
            for pattern, count in yield_frequent_patterns(new_fp_tree, new_link_table, minsup, new_freq_items):
                yield [item] + pattern, count
            
from collections import OrderedDict
def fp_growth(dataset, minsup):

    ordered_transactions, freq_items = order_transactions(dataset, minsup)
    fp_tree, link_table = build_fp_tree(ordered_transactions)
    #your code goes here
    fp_tree.saveToFile("large_fp_tree.txt")
    freq_patterns = list(yield_frequent_patterns(fp_tree, link_table, minsup, freq_items))
    for pattern, count in freq_patterns:
        pattern.sort()
    freq_patterns.sort(key=lambda pattern: (len(pattern[0]), pattern[0]))
    return freq_patterns
        
    
    
frequent_patterns = fp_growth(large_dataset, 300)
with open('fp_growth_itemsets.txt', 'w') as fi:
    print_support(frequent_patterns, len(large_dataset), fi)

## Part 3 - Extra Credit (+5 points)

**Q12.** Generate the rules having confidence above `min_conf = 0.5` using the frequent itemsets generated in Q5. Display the rules in the form of a table.

Sample table format (tab separated table)

~~~
Sup     Conf    Rule
0.44	0.67	[1] -> [2]
0.22	1.0	 [5] -> [1, 2]
0.22	1.0	 [2, 5] -> [1]
(and so on)
...
...
~~~

Note that rule confidence should be rounded to the nearest 2 decimal places (use `round(conf, 2)`). This table should also be tab (`'\t'`) separated. The rules should be displayed in the sorted order. If a rule is given as `LHS -> RHS` then the rules for which `len(LHS)` is lesser should appear first. If the `len(LHS)` is equal for two rules then rules for which `len(RHS)` is lesser should appear first. If both `len(LHS)` and `len(RHS)` is equal then the rules should be sorted based on LHS first and then based on RHS.

~~~~
Note:
LHS (Left Hand Side)
RHS (Right Hand Side)
~~~~

For eg.
~~~~
[3] -> [2] should come before [1, 3] -> [4]
[4] -> [2] should come before [2] -> [3, 4]
[1, 3] -> [2] should come before [1, 5] -> [2]
[1, 2] -> [3] should come before [1, 2] -> [5]
~~~~

Note that **this order is very important** because your output will be checked using the `diff` command. The sample output for the `small_retail.txt` dataset with `min_conf = 0.5` is:

~~~~
Sup	 Conf	Rule
0.44	0.67	[1] -> [2]
0.44	0.67	[1] -> [3]
0.44	0.57	[2] -> [1]
0.44	0.57	[2] -> [3]
0.44	0.67	[3] -> [1]
0.44	0.67	[3] -> [2]
0.22	1.0	 [4] -> [2]
0.22	1.0	 [5] -> [1]
0.22	1.0	 [5] -> [2]
0.22	1.0	 [5] -> [1, 2]
0.22	0.5	 [1, 2] -> [3]
0.22	0.5	 [1, 2] -> [5]
0.22	0.5	 [1, 3] -> [2]
0.22	1.0	 [1, 5] -> [2]
0.22	0.5	 [2, 3] -> [1]
0.22	1.0	 [2, 5] -> [1]
~~~~

**Store** this output for the `large_retail.txt` dataset in the file `apriori_rules.txt`. The sample output file for the `small_retail.txt` dataset has been provided to you as `small_apriori_rules.txt` for your convenience.

**Hint:** You shouldn't traverse the entire dataset to compute the confidence for a rule since you have already computed the `support_data` for all the frequent itemsets. 

In [17]:
# your code goes here

def calculate_confidence(antecedent, itemset, support_count):
    return support_count[itemset] / support_count[antecedent]

def generate_rules_for_itemset(itemset, min_conf, support_count):
    rules_generated = []
    for item in itemset:
        consequent = frozenset([item])
        antecedent = itemset - consequent
        confidence = calculate_confidence(antecedent, itemset, support_count)
        if confidence >= min_conf:
            rules_generated.append((antecedent, consequent))
            yield (antecedent, consequent, confidence)
    
    while rules_generated:
        prev_generated_rules = rules_generated
        rules_generated = []
        for index, rule in enumerate(prev_generated_rules):
            antecedent, consequent = rule
            if len(antecedent) == 1:
                break
            for paired_antecedent, paired_consequent in prev_generated_rules[index + 1:]:
                new_cons = consequent | paired_consequent
                if len(new_cons) > len(consequent) + 1:
                    continue
                else:
                    assert len(new_cons) == len(consequent) + 1
                    new_ant = itemset - new_cons
                    assert len(new_ant) == len(antecedent) - 1
                    confidence = calculate_confidence(new_ant, itemset, support_count)
                    if confidence >= min_conf:
                        rules_generated.append((new_ant, new_cons))
                        yield(new_ant, new_cons, confidence)
        
        
def generate_rules(freq_itemsets, min_conf, support_count):
    rules = []
    for itemset in freq_itemsets:
        if len(itemset) >= 2:
            itemset_rules = list(generate_rules_for_itemset(itemset, min_conf, support_count))
            rules += itemset_rules
    return rules

frequent_patterns, support_count = apriori_freq_itemsets(large_dataset, 300)
rules = generate_rules(frequent_patterns, 0.5, support_count)

def format_rules(rules, support_count):
    ordered_rules = []
    for antecedent, consequent, min_conf in rules:
        itemset = antecedent | consequent
        ordered_antecedent = sorted(antecedent)
        ordered_consequent = sorted(consequent)
        ordered_rules.append((ordered_antecedent, ordered_consequent, min_conf, support_count[itemset]))
    ordered_rules.sort(key=lambda rule: (len(rule[0]), rule[0], len(rule[1]), rule[1]))
    return ordered_rules
    
formatted_rules = format_rules(rules, support_count)

def print_rules(formatted_rules, num_transactions, fi):
    print("Sup\tConf\tRule", file=fi)
    for ant, cons, conf, sup in formatted_rules:
        sup /= num_transactions
        print("{}\t{}\t{} -> {}".format(round(sup, 2), round(conf, 2), ant, cons), file=fi)

with open("apriori_rules.txt", "w") as fi:
    print_rules(formatted_rules, len(large_dataset), fi)
            